In [23]:
#We need these in this file:
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import numpy as np # linear algebra
import get_lodging_scores
import load_read_name_extractor as lrne
import SVM_classifier_general as svm_general
import pickle
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV


In [24]:
#Iteration
#hog
hog_cell = "8x8"
hog_block = "2x2"
image_size = "64_128"

#svm
bin_size = 33
n_bins = int(np.floor(100/bin_size))
class_weights = "none" #removed
svm_kernel = "rbf"
scorer_function = "dist"

# Define parameter grid for grid search
param_grid = {'C': np.logspace(-3, 3, 6), 'gamma': np.logspace(-3, 1, 3)}
optimize_params = "C_and_gamma"


In [25]:
features_load = np.load('features/hog_features_'+image_size+'_hog_cells_'+hog_cell+'_block_'+hog_block+'.npy', allow_pickle=True)
names_load = np.load('features/img_names_'+image_size+'_hog_cells_'+hog_cell+'_block_'+hog_block+'.npy', allow_pickle=True)
print(len(features_load))
print(len(names_load))

4935
4935


In [26]:
labels = get_lodging_scores.get_labels(bin_size)
#[ROI, Visual SCORE, FLIGHTFolder, BIN]
print(labels[0])

(61, 50, '160607_Min', 1)


In [27]:
my_set = set(map(tuple, labels))
# convert set back to list of arrays
labels = list(map(np.array, my_set))

In [28]:
names = lrne.it_name_extract_labels_from_img_jpeg(names_load)

In [29]:
clf = svm.SVC(kernel=svm_kernel)
features_aar = np.array(features_load)
data_frame, error = svm_general.match_pic_label_to_names(features_load, labels, names)
#data_frame_names, error_names = match_pic_label_names(features_load, labels, names)
np.random.shuffle(data_frame)
data_frame = np.array(data_frame)


In [30]:
percentage = 80
partition = int(len(features_aar)*percentage/100)
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
print(y_train)


['1' '1' '1' ... '1' '0' '0']


In [31]:
clf.fit(x_train,y_train)

SVC()

In [32]:
y_pred = clf.predict(x_test)

In [33]:
filename = 'model/SVM_'+svm_kernel+'_'+image_size+'_'+str(n_bins)+'_cells_'+hog_cell+'_block_'+hog_block+'_'+class_weights+'.sav'
pickle.dump(clf, open(filename, 'wb'))

In [34]:
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [35]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.6163391933815926


              precision    recall  f1-score   support

           0       0.53      0.31      0.39       308
           1       0.64      0.83      0.72       571
           2       0.59      0.31      0.40        88

    accuracy                           0.62       967
   macro avg       0.59      0.48      0.51       967
weighted avg       0.60      0.62      0.59       967



In [36]:
# Define cost matrix
num_classes = n_bins
cost_matrix = [[0]*num_classes for i in range(num_classes)]

for i in range(num_classes):
    for j in range(num_classes):
        cost_matrix[i][j] = abs(i-j)


In [37]:
cost_matrix


[[0, 1, 2], [1, 0, 1], [2, 1, 0]]

In [38]:
def custom_scorer(y_true, y_pred):
    """
    Calculates the cost of classifying class i as class j.
    :param y_true: true labels
    :param y_pred: predicted labels
    :return: total cost
    """
    cost = 0
    for i in range(len(y_true)):
        cost += cost_matrix[int(y_true[i])][int(y_pred[i])]
    return cost

# Create scorer
scorer = make_scorer(custom_scorer, greater_is_better=False)

In [39]:
# Define SVM model
svm = clf
# Perform grid search using custom scorer
grid_search = GridSearchCV(svm, param_grid, scoring=scorer)
grid_search.fit(x_train,y_train)


GridSearchCV(estimator=SVC(),
             param_grid={'C': array([1.00000000e-03, 1.58489319e-02, 2.51188643e-01, 3.98107171e+00,
       6.30957344e+01, 1.00000000e+03]),
                         'gamma': array([1.e-03, 1.e-01, 1.e+01])},
             scoring=make_scorer(custom_scorer, greater_is_better=False))

In [40]:
results = pd.DataFrame(grid_search.cv_results_)
params = results[['params']]
params = pd.DataFrame(params['params'].values.tolist())

scores = results[['mean_test_score', 'std_test_score']]
results_df = pd.concat([params, scores], axis=1)
print(results_df)

              C   gamma  mean_test_score  std_test_score
0      0.001000   0.001           -365.6        0.489898
1      0.001000   0.100           -365.6        0.489898
2      0.001000  10.000           -365.6        0.489898
3      0.015849   0.001           -365.6        0.489898
4      0.015849   0.100           -365.6        0.489898
5      0.015849  10.000           -365.6        0.489898
6      0.251189   0.001           -365.6        0.489898
7      0.251189   0.100           -365.6        0.489898
8      0.251189  10.000           -365.6        0.489898
9      3.981072   0.001           -364.4        1.200000
10     3.981072   0.100           -338.8        5.564171
11     3.981072  10.000           -365.6        0.489898
12    63.095734   0.001           -378.8        3.655133
13    63.095734   0.100           -338.8        5.564171
14    63.095734  10.000           -365.6        0.489898
15  1000.000000   0.001           -409.0       11.207141
16  1000.000000   0.100        

In [41]:

# Print best parameters and score
print(grid_search.best_params_)
print(grid_search.best_score_)

{'C': 3.981071705534969, 'gamma': 0.1}
-338.8


In [42]:
filename = 'model/SVM_'+svm_kernel+'_'+image_size+'_'+str(n_bins)+'_cells_'+hog_cell+'_block_'+hog_block+'_'+class_weights+'_scorer_'+scorer_function+'_'+optimize_params+'_.sav'

pickle.dump(grid_search, open(filename, 'wb'))

In [43]:
y_pred = grid_search.predict(x_test)
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.6132368148914168


              precision    recall  f1-score   support

           0       0.52      0.37      0.43       308
           1       0.65      0.79      0.71       571
           2       0.59      0.30      0.39        88

    accuracy                           0.61       967
   macro avg       0.58      0.49      0.51       967
weighted avg       0.60      0.61      0.59       967



In [44]:
print(y_pred)

['2' '1' '0' '0' '0' '0' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1' '0'
 '1' '0' '1' '1' '1' '1' '1' '0' '0' '1' '0' '1' '0' '0' '0' '0' '2' '1'
 '0' '1' '1' '0' '0' '1' '1' '0' '0' '1' '1' '1' '1' '1' '1' '0' '1' '1'
 '1' '1' '1' '0' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1' '1' '1' '1' '0' '1' '1' '1'
 '1' '1' '0' '1' '0' '1' '1' '1' '1' '0' '1' '0' '1' '0' '1' '1' '0' '1'
 '1' '0' '1' '1' '1' '1' '1' '1' '1' '1' '2' '1' '1' '1' '1' '0' '2' '1'
 '1' '1' '0' '1' '0' '1' '1' '2' '1' '1' '1' '1' '1' '2' '0' '0' '1' '2'
 '0' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '2' '0' '1' '1' '1' '2' '1' '1' '1' '1' '1' '0' '1' '1' '0'
 '1' '1' '1' '1' '2' '1' '1' '2' '1' '1' '1' '0' '1' '1' '1' '1' '1' '1'
 '0' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1' '2' '0' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1' '1' '0' '1' '1'
 '0' '1' '1' '1' '0' '1' '1' '1' '1' '1' '1' '1' '1